In [9]:
import astropy.io.fits as pyfits
import k3d
import numpy as np
import matplotlib.cm
import time
import pylab as pl

In [15]:
# decide which map you want to load

#maps = pyfits.open('3dmap_XYZvel.fits')[0].data ; PERC = 0 ; cmap = matplotlib.cm.Spectral_r ; imtype = 'vel'
#maps = pyfits.open('3dmap_XYZflux.fits')[0].data ; PERC = 10 ; cmap = matplotlib.cm.viridis ; imtype = 'flux'
#maps = pyfits.open('3dmap_XYZnii_ha.fits')[0].data ; PERC = 10 ; cmap = matplotlib.cm.viridis ; imtype = 'nii_ha'
#maps = pyfits.open('3dmap_XYZsii_ha.fits')[0].data ; PERC = 10 ; cmap = matplotlib.cm.viridis ; imtype = 'sii_ha'
maps = pyfits.open('3dmap_XYZsii_sii.fits')[0].data ; PERC = 6 ; cmap = matplotlib.cm.viridis ; imtype = 'sii_sii'


In [17]:
# display data
filtermaps = pyfits.open('3dmap_XYZflux.fits')[0].data[:,3] 

positions = maps[:,:3]
colors = maps[:,3]
if imtype != 'sii_sii':
    colors[np.nonzero(filtermaps < np.nanpercentile(filtermaps, 10))] = np.nan
cbar_min, cbar_max = np.nanpercentile(colors, [PERC, 100-PERC])

colors -= np.nanpercentile(colors, PERC)
colors /= np.nanpercentile(colors, 100-PERC)

colors[colors < 0] = 0
colors[colors > 1] = 1

positions = positions[~np.isnan(colors)]
colors = colors[~np.isnan(colors)]

rgbacolors = (cmap(colors) * 255).astype(np.uint32)
intcolors = rgbacolors[:,0]
intcolors = rgbacolors[:,0] << 16 | rgbacolors[:,1] << 8 | rgbacolors[:,2]

RADIUS = 4
FOV = 50
PREFIX = ''

def rotate_camera(plot, angle, image_number):
    radius = RADIUS # 4
    z = np.cos(np.deg2rad(angle)) * radius
    x = np.sin(np.deg2rad(angle)) * radius
    plot.grid = [-1, -1, -1, 1, 1, 1]
    # camera [posx,posy,posz,targetx,targety,targetz,upx,upy,upz]
    plot.camera = [x,0,z,0,0,0,0,1,0]
    plot.camera_fov = FOV # 50
    
plot = k3d.plot(height=800)
plot.camera_auto_fit = False
plot.grid_auto_fit = True

points = k3d.points(positions.astype(np.float32), colors=intcolors, point_size=1.5, shader='dot', opacity=1., legend=True, )
plot += points
plot.display()
plot.camera = [0,0,2.8,0,0,0,0,0,0]
angle = 0
image_number = 0
    
rotate_camera(plot, angle, image_number)


Output()